In [1]:
import os
import sys
import yaml

from torch import zeros

project_root = os.path.abspath(os.path.join(os.getcwd(), "../../.."))
sys.path.append(project_root)

from utils.data_processing import get_finance_dataset

In [2]:
# Generate the dataset and dataloaders
path = os.path.join(os.getcwd(), "data/")

train_frac = 0.4
valid_frac = 0.0
test_frac = 0.3

dataset, train_indices, validation_indices, test_indices = get_finance_dataset(path,
                                                                            train_frac ,
                                                                            valid_frac,
                                                                            test_frac)

Loading dataset...
Loaded dataset from /home/fazeleh/LeakPro/examples/mia/DT/data/dataset.pkl


In [3]:
import numpy as np
unique, counts = np.unique(dataset.y, return_counts=True)
counts_dict = dict(zip(unique, counts))
print(counts_dict)

{0.0: 98325, 1.0: 1662}


In [4]:
from utils.model_XGBoost import XGBoostModel, create_trained_model_and_metadata

n_classes = 2
n_estimators = 100

model = XGBoostModel(n_classes, n_estimators)

# Load the config.yaml file
with open('train_config.yaml', 'r') as file:
    train_config = yaml.safe_load(file)

train_acc, test_acc = create_trained_model_and_metadata(model,
                                                        dataset,
                                                        train_indices,
                                                        validation_indices,
                                                        test_indices,
                                                        train_config)


Training XGBoost model...


/home/fazeleh/miniconda3/envs/leakpro_py311/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [15:01:03] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1738880369036/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Training Accuracy: 0.9999, Test Accuracy: 0.9845


##  Attacking the XGBoost model

In [5]:
from DT_handler import DTInputHandler
from leakpro import LeakPro

# Read the config file
config_path = "audit.yaml"

# Prepare leakpro object
leakpro = LeakPro(DTInputHandler, config_path)

# Run the audit
mia_results = leakpro.run_audit(return_results=True)

2025-03-05 15:01:08,692 INFO     Target model blueprint created from XGBoostModel in utils/model_XGBoost.py.
2025-03-05 15:01:08,719 INFO     Loaded target model metadata from ./target/model_metadata.pkl
2025-03-05 15:01:08,723 INFO     Loaded target model from ./target
2025-03-05 15:01:08,759 INFO     Loaded population dataset from ./data/dataset.pkl
2025-03-05 15:01:08,760 INFO     Loaded population dataset from ./data/dataset.pkl
/home/fazeleh/miniconda3/envs/leakpro_py311/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/fazeleh/miniconda3/envs/leakpro_py311/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
2025-

AssertionError: Batch size must be greater than 0